<a href="https://colab.research.google.com/github/CosmicSelection/DuelSim/blob/Mahdsiah/DuelSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install yugioh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import random as rand
import re
import yugioh

class onField:
  def __init__(self, card, pos, isFaceUp):
    self.pos = pos
    self.isFaceUp = isFaceUp
    self.name = card.name
    self.type = card.type
    self.description = card.description
    cardType = re.search("monster", card.type.lower())
    if cardType != None:
      self.attack = card.attack
      self.bonus_atk = 0
      self.total_atk = self.attack + self.bonus_atk
      self.defense = card.defense
      self.bonus_def = 0
      self.total_def = self.defense + self.bonus_def
      self.race = card.race
      self.level = card.level
      self.attribute = card.attribute
      self.posChanged = True
      self.equipped = []
      self.can_atk = True
      self.switched = False
    
   

  def __str__(self):
    cardType = re.search("monster", card.type.lower())
    if cardType == None:
      return f"{self.name} ({self.type}: {self.description})"
    return f"{self.name} ({self.attack} atk, {self.defense} def)"
  
def atk_pts(monster):
  return monster.total_atk

def def_pts(monster):
  return monster.total_def



In [47]:



p1DeckBuild = np.array(["Mystical Elf", "Winged Dragon, Guardian of the Fortress #1", "Summoned Skull", "Beaver Warrior", "Dark Magician",
                        "Gaia The Fierce Knight", "Curse of Dragon", "Celtic Guardian", "Mammoth Graveyard", "Silver Fang",
                        "Giant Soldier of Stone", "Dragon Zombie", "Doma The Angel of Silence", "Ansatsu", "Witty Phantom", "Claw Reacher", 	
                        "Sword of Dark Destruction", "Book of Secret Arts", "Dark Hole", "Dian Keto the Cure Master", "Ancient Elf",
                        "Magical Ghost", "Fissure", "Trap Hole", "De-Spell", "Monster Reborn", "Reinforcements", "Change of Heart",
                        "Wall of Illusion", "Neo the Magic Swordsman", "Baron of the Fiend Sword", "Sorcerer of the Doomed", 	"Waboku", "Dian Keto the Cure Master", 
                        "Magic Cylinder", "Sakuretsu Armor", "Remove Trap", "Castle Walls", "Ultimate Offering", "Raigeki"])
                        
p2DeckBuild = np.array(["Blue-Eyes White Dragon", "Hitotsu-Me Giant", 	"Battle Ox", 	"Koumori Dragon", "Judge Man", 
                        "Rogue Doll", "Kojikocy", "Uraby", "Gyakutenno Megami", "Mystic Horseman", "Terra the Terrible", "Dark Titan of Terror", 
                        "Master & Expert", 	"Mystic Clown", "Ogre of the Black Shadow", "Dark Energy", "Invigoration", "Dark Hole",
                        "Ookazi", "Ryu-Kishin Powered", "Swordstalker", "La Jinn the Mystical Genie of the Lamp", "Rude Kaiser", "Destroyer Golem",
                        "Skull Red Bird", "D. Human", "Pale Beast", "Fissure", "Trap Hole", "Mystical Space Typhoon", 	"De-Spell", "Castle Walls",
                        "Monster Reborn", "Reinforcements", "Raigeki", "Ultimate Offering", "Galaxy Serpent", "Dian Keto the Cure Master" ,
                        "Sakuretsu Armor", "Remove Trap"])


p1Deck = []
p2Deck = []

for name in p1DeckBuild:
  card = yugioh.get_card(card_name = name)
  p1Deck.append(card)


for name in p2DeckBuild:
  card = yugioh.get_card(card_name = name)
  p2Deck.append(card)
  



In [ ]:
#Main
def letsDuel(p1Deck, p2Deck):
  #Field Structure
  # p1 spell/trap zone -> [None, None, None, None, None]  Field[0][0]
  # p1 monster zone -> [None, None, None, None, None]     Field[0][1]
  # p2 monster zone -> [None, None, None, None, None]     Field[1][0]
  # p2 spell/trap zone -> [None, None, None, None, None]  Field[1][1]
  Field = [[[], []], [[], []]]

  #grave for all used or destroyed cards
  p1GY = []
  p2GY = []

  #player
  p1 = 1
  p2 = 2   

  #shuffle  decks 
  rand.shuffle(p1Deck)
  rand.shuffle(p2Deck)

  #initialize start hands
  p1Hand= []
  p2Hand= []
  for i in range(5):
    draw(p1, p1Deck, p1Hand, p1GY)
    draw(p2, p2Deck, p2Hand, p2GY)

  #life points
  p1LP = 4000
  p2LP = 4000
  player = rand.randrange(1,3)
  print("Player " + player + " Starts")
  while (p1LP > 0) and (p2LP > 0) and (len(p1Deck) > 0) and (len(p2Deck) > 0):
    if player == 1:
      print("Player 1's Turn")
      print()
      draw(player, p1Deck, p1Hand, p1GY )
      print("Standby Phase")
      print()
      p1LP, p2LP = strategy(player, p1Hand, Field, p1GY, p2GY, p1LP, p2LP)
      player = p2
    else:
      print("Player 2's Turn")
      print()
      draw(player, p2Deck, p2Hand, p2GY )
      print("Standby Phase")
      print()
      p2LP, p1LP = strategy(player, p2Hand, Field, p2GY, p1GY, p2LP, p1LP)
      player = p1
    print()
  if (p1LP == 0 and p2LP == 0) or (len(p1Deck) == 0 and len(p2Deck) == 0):
    print("TIED GAME")
  elif p2LP == 0 or len(p2Deck) == 0:
    print("Player 1 Wins!")
  elif p1LP == 0 or len(p1Deck) == 0:
    print("Player 2 Wins!") 






In [62]:
#All Game Mechanics


#Summoning Mechanics
def normalSummon(player, pHand, Field, monster):
   if player == 1:
      x = 0
      y = 1
   else:
      x = 1
      y = 0
   if len(Field[x][y]) < 5:
    newMon = onField(monster, "attack", True)
    print("Summon", newMon)
    Field[x][y] = np.append(Field[x][y], newMon)
    pHand.remove(monster)

def flipSummon(player, Field, monsters):
  if player == 1:
    x = 0
    y = 1
  else:
    x = 1
    y = 0
  for i in range(len(Field[x][y])):
    for monster in monsters:
      if Field[x][y][i].name == monster.name:
        print("Flip Summon", monster)
        Field[x][y][i].pos = "attack"
        Field[x][y][i].isFaceUp = True
        break
    
def setMonster(player, pHand, Field, monster):
    if player == 1:
      x = 0
      y = 1

    else:
      x = 1
      y = 0
     
    if len(Field[x][y]) < 5:
      print("Set 1 monster Face Down")
      newMon = onField(monster, "defense", False)
      Field[x][y].append(newMon)
      pHand.remove(monster)

def setTrap(player, pHand, Field, traps):
    if player == 1:
      x = 0
      y = 0
    else:
      x = 1
      y = 1
    trap_counter = 0
    if Field[x][y]:
      for card in Field[x][y]:
        if "Trap" in card.type:
          trap_counter += 1
    set_counter = 3 - trap_counter
    if set_counter > 0:
      delete = []
      print("Set", set_counter, "Trap(s)")
      for i in range(len(traps)): 
        if i < set_counter:
          newTrap = onField(traps[i], None, False)
          Field[x][y].append(newTrap)
          delete.append(traps[i])
      for card in delete:
        pHand.remove(card)

#Game Phases
def draw(player, deck, hand, GY):
  print()
  print("Draw Phase")
  print()
  while len(hand) >= 7:
    minMon = None
    min = float('inf')
    counter = 0
    for card in hand: 
      if "Monster" in card.type:   
        counter += 1
        if card.attack < min:
          min = card.attack
          minMon = card
        elif card.attack == min and card.defense > minMon.defense:
          minMon = card
    if counter < 3:
      i = rand.randrange(len(hand))
      if "Monster" not in hand[i].type:
        discard = hand.pop(i)
        print("Too Many Cards In Hand: Discard", discard.name)
        GY.append(discard)
    else:
      print("Too Many Cards In Hand: Discard", minMon.name)
      hand.remove(minMon)
      GY.append(minMon)
  draw = deck.pop(0)
  print("Draw", draw.name)
  hand.append(draw)

def main1(player, pHand, Field, GY, opGY, LP, opLP, main1_params, battle_params, main2_params, end_phase):
  print()
  print("Main Phase")
  print()
  LP, opLP = battle(player, Field, pHand, GY, opGY, LP, opLP, battle_params, main2_params, end_phase)
  return LP, opLP
  
def battle(player, Field, pHand, GY, opGY, LP, opLP, battle_params, main2_params, end_phase):
  print()
  print("Battle Phase")
  print()
  main2(player, Field, pHand, main2_params, end_phase)
  return LP, opLP

def main2(player, Field, pHand, main2_params, end_phase):
    print()
    print("Main Phase 2")
    print()
    hand_traps = main2_params["hand_traps"]
    change_pos = main2_params["change_pos"]
    set_monster = main2_params["set"]
    if change_pos:
        changePosition(player, Field, change_pos)
    if hand_traps:
        setTrap(player, Field, hand_traps)
    if set_monster != None:
          setMonster(player, pHand, Field, set_monster)


    end(player, Field, end_phase)
    
 
def end(player, Field, end_phase):
    print()
    print("End Phase")
    print()
    if player == 1:
        x = 0
        y = 1
    elif player == 2:
        x = 0
        y = 1
    # Check if end_phase is empty
    if end_phase:
        #Check if end_phase contain Reinforcement or Castle Walls or Change of Hearts before checking each card(Probably Dont need to Check)
        if any(card == "Reinforcements" or card == "Castle Walls" or card == "Change Of Hearts" for card in end_phase):
            #Go to each card in player field
            for card in Field[x][y]:
                if card.equipped:
                    for equipped_card in card.equipped:
                        if equipped_card == "Reinforcements":
                            print("Reinforcements Effect Ends: " + card.name + " Loses 500 Atk")
                            card.equipped.remove(equipped_card)
                            card.bonus_atk -= 500
                        if equipped_card == "Castle Walls":
                            print("Castle Walls Effect Ends: " + card.name + " Loses 500 Def")
                            card.equipped.remove(equipped_card)
                            card.bonus_def -= 500
                #Change of Heart Implementation
                if card.switched:
                    #Send Opponent back to their Field
                    print("Change Of Heart Effect Ends: " + card.name + " Goes Back to Opponents Field")
                    Field[y][x].append(card)
                    Field[x][y].remove(card)
                    card.switched = False
     
    print("End Turn")
    
    

#Other Game mechanics
def changePosition(player, Field, monsters):
  if player == 1:
    x = 0
    y = 1
  else:
    x = 1
    y = 0
  if monsters[0].pos == 'attack':
    print("Switching All Monsters To Defense Position")
  for i in range(len(Field[x][y])):
    for monster in monsters:
      if Field[x][y][i].name == monster.name:
        if Field[x][y][i].pos == "attack":
          print( monster, "Switched to Defense Position")
          Field[x][y][i].pos = 'defense'
          break
        else:
          print(monster, "Switched to Attack Position")
          Field[x][y][i].pos = "attack"
          break
   

def attack(player, Field, GY, opGY, LP, opLP, attacking, attacked):
  if player == 1:
    x = 0
    y = 1
    notPlayer = 2
  else:
    x = 1
    y = 0
    notPlayer = 1
  if attacked == None:
    opLP -= attacking.total_atk
  elif attacked.pos == "attack":
    lpLoss = attacking.total_atk - attacked.total_atk
    print(attacking, "Attacks Attack Position", attacked)
    if lpLoss > 0:
      print("Attack Successful")
      print(attacked.name, "Destroyed")
      print("Opponent Loses", lpLoss, "Life Points" )
      opLP -= (lpLoss)
      destroy(notPlayer, Field, opGY, attacked)
      
    elif lpLoss < 0:
      print("Attack Failed")
      print(attacking.name, "Destroyed")
      print("I Lose ", -lpLoss, "Life Points" )
      LP += (lpLoss)
      destroy(player, Field, GY, attacking)
    else:
      print("Attack Semi-Successful")
      print("Both Monsters Destroyed")
      destroy(player, Field, GY, attacking)
      destroy(notPlayer, Field, opGY, attacked)
  else:
    if not attacked.isFaceUp:
      attacked.isFaceUp = True
    print(attacking, "Attacks Defense Position", attacked)
    if attacked.total_def >= attacking.total_atk:
      lpLoss = attacked.total_def - attacking.total_atk
      print("Attack Failed")
      print("I Lose", lpLoss, "Life Points")
      LP -= (lpLoss)
    else:
      print("Attack Successful")
      print(attacked.name, "Destroyed")
      destroy(notPlayer, Field, opGY, attacked)
  
  return LP, opLP

def destroy(player, Field, GY, destroyed):
  if player == 1:
    x = 0
    y = 1
  else:
    x = 1
    y = 0

  Field[x][y].remove(destroyed)
  destroyed.bonus_atk = 0
  destroyed.bonus_def = 0
  if destroyed.equipped:
    for equip in destroyed.equipped:
      for card in Field[x][x]:
        if card.name == equip:
          Field[x][x].remove(card)
          break    
  destroyed.equipped = []
  GY.append(destroyed)

def activate(player, turn_player, Field, GY, opGY, Lp, opLP, phase):
  return LP, opLP


def strategy(player, pHand, Field, GY, opGY, LP, opLP):
  print("Strategizing. Please Wait... ")
  temp_field = Field
  temp_hand = pHand
  hand_monsters = []
  useful_spells = {}
  field_spells = {}
  useful_traps = {}
  hand_traps = {}
  end_phase = []
  field_test = False
  equip_spells = [ "Invigoration", "Sword of Dark Destruction", "Book of Secret Arts", "Dark Energy"]

  if player == 1:
    x = 0
    y = 1
  else:
    x = 1
    y = 0

  for card in pHand:
    if 'monster' in card.type.lower():
      hand_monsters.append(card)
    if 'spell' in card.type.lower() and (card.name not in equip_spells):
      useful_spells[card.name] = activate(card.name)
    if 'trap' in card.type.lower():
      if len(Field[x][x]) < 3:
        hand_traps.append(card)
  
  for card in Field[x][x]:
    useful_traps[card.name] = activate(card.name)


  if "Reinforcements" in useful_traps.keys() and useful_traps["Reinforcements"] == True:
    end_phase.append("Reinforcements")
  if "Change Of Heart" in useful_spells.keys() and useful_spells["Change Of Heart"] == True:
    end_phase.append("Change Of Heart")

  optimal_monster_on_field = max(temp_field[x][y], key=lambda monster: monster.total_atk)
  optimal_monster_in_hand = max(hand_monsters, key = lambda monster: monster.attack)
  in_hand = optimal_monster_in_hand.attack
  on_field = optimal_monster_on_field.total_atk

  if on_field < in_hand and len(temp_field[x][y]) < 5:
    normalSummon(player, temp_hand, temp_field, monster)
    z = len(temp_field[x][y])-1
    optimal_monster =  temp_field[x][y][z]
    field_test = True 
  else:
     optimal_monster = optimal_monster_on_field


  for card in pHand:
    if (card.name in equip_spells):
      field_spells[card.name] = activate(card.name) #equip spell to optimal monster on temp_field



  op_optimal_monster = max(temp_field[y][x], key=lambda monster: monster.total_atk)
  optimal_monster = max(temp_field[x][y], key=lambda monster: monster.total_atk)

  min_monster = min(monsters, key = lambda monster: monster.total_atk)
  monsters = []
  for monster in Field[x][y]:
    if monster.pos == "attack":
      monsters.append(monster)
  monsters.sort(reverse = True, key = atk_pts)
  

  op_monsters_atk = []
  for monster in Field[y][x]:
    if monster.pos == "attack":
      op_monsters_atk.append(monster)
  op_monsters_atk.sort(reverse = True, key = atk_pts)

  op_monsters_def = []
  for monster in Field[y][x]:
    if monster.pos == "defense" and monster.isFaceUp == True:
      op_monsters_def.append(monster)
  op_monsters_def.sort(reverse = True, key = def_pts)
  for monster in Field[y][x]:
    if monster.pos == "defense" and monster.isFaceUp == False:
      op_monsters_def.append(monster)

  op_monsters = op_monsters + op_monsters_def

  if op_optimal_monster.total_atk > optimal_monster.total_atk:
    main1_params = {"useful_spells": useful_spells, "useful_traps": useful_traps, "summon": None, "change_pos": [] }
    battle_params = {"monsters": monsters, "op_monsters": op_monsters}
    main2_params = {"hand_traps": hand_traps, "change_pos": monsters, "set": min_monster}
    main1(player, pHand, Field, GY, opGY, main1_params, battle_params, main2_params, end_phase)
    

  else: 
    if len(temp_field[y][y]) > 2:
      hand_monsters.remove(optimal_monster_in_hand) 
      optimal_monster_in_hand2 = max(hand_monsters, key = lambda monster: monster.attack)
      optimal_points2 = max(optimal_monster_in_hand2.attack, on_field)
      if optimal_points2 > op_optimal_monster.total_atk and field_test:
        temp_hand.append(temp_field.pop())
        normalSummon(player, temp_hand, temp_field, optimal_monster_in_hand2)
    useful_spells.update(field_spells)
    attack_wins = []
    i = 0
    if field_test:
      z = len(temp_field[x][y]) - 1
      monsters.append(temp_field[x][y][z])
    for monster in monsters:
      if i < len(op_monsters):
        op = op_monsters[i]
        if op.pos == "attack": 
          if monster.total_atk > op.total_atk:
            attack_wins.append(True)
          else:
            attack_wins.append(False)
        else:
          if monster.total_atk > op.total_def:
            attack_wins.append(True)
          else:
            attack_wins.append(False)
      else:
        break
      i = i + 1

    monsters_def = []
    for monster in Field[x][y]:
      if monster.pos == "defense":
        monsters_def.append(monster)
    monsters_def.sort(reverse = True, key = atk_pts)
    i = 0
    change_pos = []
    for monster in monsters_def:
      if i < len(op_monsters):
        op = op_monsters[i]
        win = attack_wins[i]
        if not win and op.total_atk < monster.total_atk:
          change_pos.append(monster)
          monsters.append(monster)
        else:
          break
      else:
        break
      i = i + 1
    monsters.sort(reverse = True, key = atk_pts)
    main1_params = {"useful_spells": useful_spells, "useful_traps": useful_traps, "summon": optimal_monster_in_hand, "change_pos": change_pos }
    battle_params = {"monsters": monsters, "op_monsters": op_monsters}
    main2_params = {"hand_traps": hand_traps, "change_pos": [], "set": None}
    LP, opLP = main1(player, pHand, Field, GY, opGY, LP, opLP, main1_params, battle_params, main2_params, end_phase )
    
    return LP, opLP


def counter_strategy(player, pHand, Field, phase):
  pass

  

    



In [56]:
#Card Effects

# examples = 'Monster Reborn', 'Remove Trap', 'Raigeki' 
#(cards return None if the activation cost can't be met, or they are useless)

#cards that still need to be defined
#set 1: Sword of Dark Destruction', 'Book of Secret Arts', 'Dark Hole', 'Dian Keto the Cure Master' 
#set 2: 'Fissure', 'Trap Hole', 'De-Spell', 'Reinforcements'
#set 3: 'Change of Heart', 'Waboku', 'Soul Exchange', 'Dragon Capture Jar'
#set 4: 'Castle Walls', 'Invigoration', 'Sakuretsu Armor', 'Dark Energy'
#set 5: 'Ultimate Offering', 'Ookazi', 'Mystical Space Typhoon', 'Magic Cylinder'

def magicCylinder(player, opAttackingCard, opLP):
  opLP -= opAttackingCard.total_atk
  return opLP

def ookazi(opLP):
  print("Opponent's life points decreased by 800")
  opLP -= 800
  return opLP

def mysticalSpaceTyphoon(player, Field, opGY):
  if player == 1:
    x = y = 1
  else:
    x = y = 0
  
  if len(Field[x][y]) > 0:
    print(Field[x][y][0].name + " destroyed" )
    opGY.append(Field[x][y][0])
    Field[x][y].pop(0)
    return True
  else:
    return False
def ultimateOffering(player, pHand, Field, pLP):
  max = 0
  maxMon = None
  min = float('inf')
  minMon = None
  if player == 1:
    x = 0
    y = 1
  else: 
    x = 1
    y = 0


  if len(Field[x][y]) <= 5 and len(pHand) > 0 and pLP > 500:
    for monster in pHand:
      cardType = re.search("monster", card.type.lower())
      if cardType != None:
        if max <= monster.attack:
          max = monster.attack
          maxMon = monster
        if min > monster.attack:
          min = monster.attack
          minMon = monster
    if maxMon == None and minMon == None:
      print("No monsters to summon")
      return False
    max2 = max
    for monster in Field[x][y]:
      if monster.total_atk> max2:
        max2 = monster.total_atk
    if len(Field[y][x]) > 0:
      for monster in Field[y][x]:
        if max2 < monster.total_atk:
          print("Pay 500 life points")
          pLP = pLP - 500
          print("Set " + minMon.name)
          setMonster(player, pHand, Field, minMon)
          return True
    print("Pay 500 life points")
    pLP = pLP - 500
    print("Summon " + maxMon.name)
    normalSummon(player, pHand, Field, maxMon)
    return True
  else:
    return False

#special someone monster from either GY
def monsterReborn(player, Field, GY, opGY):
  max = 0
  maxMonster = None

  #initialize player field
  if player == 1:
    x = 0
    y = 1
  elif player == 2:
    x = 1
    y = 0
  
  #if either Graveyard has cards and your field isnt full
  if len(Field[x][y]) <= 5:
    op = False

    # search for best monster in either graveyard
    if len(GY) > 0:
      for card in GY:
          cardType = re.search("monster", card.type.lower())
          if (cardType != None) and (card.attack > max):
            max = card.attack
            maxMonster = card
    if len(opGY) > 0:
      for card in opGY:
          cardType = re.search("monster", card.type.lower())
          if (cardType != None) and (card.attack > max):
            max = card.attack
            maxMonster = card
            op = True

    if maxMonster != None and op == False:
      GY.remove(maxMonster)
    elif maxMonster != None and op == True:
      
      opGY.remove(maxMonster)

  #if you can't summon a monster return None
    else:
      print("There are no monsters to special summon")
      return False

    #if there exists such a monster determine if it is stronger than opponents monsters
    opMax = max(Field[y][x], key = lambda monster: monster.total_attack)
    if opMax.total_attack > maxMonster.attack:
      pos = 'defense'
    else:
      pos = 'attack'
    summon = onField(maxMonster, pos, True)
    print("Special Summon " + summon + " from the graveyard")
    Field[x][y].append(summon)
    return True
  else:
    print("There are no monsters to special summon")
    return False

def removeTrap(player, Field, opGY):
  if player == 1:

    #set (x, y) values to opposite oppenents spell/trap zone
    x = y = 1

  elif player == 2:

    #set (x, y) values to opposite oppenents spell/trap zone
    x = y = 0 

  trap = None

  #if opponent has a spell or trap on the field
  if len(Field[x][y]) > 0:

    #search for a trap to destroy
    for card in Field[x][y]:
        cardType = re.search("trap", card.type.lower())
        if (cardType != None):
          trap = card
          break
    
    #destroy trap
    if trap == None:
      print("No traps to destroy")
      return False
    opGY.append(trap)
    Field[x][y].remove(trap)
    print(trap.name + " destroyed")
    return True
  else:
    print("No cards in spell/trap zone")
    #return None if no traps can be destroyed
    return False

def raigeki(player, Field, opGY):
  
  #set field (x, y) values to opposite players field
  if player == 1:
    x = 1
    y = 0
    notPlayer = 2
  else:
    x = 0
    y = 1
    notPlayer = 1

  if len(Field[x][y]) > 0:
    
    #destroy each monster on the opponents field
    for mon in Field[x][y]:
      opGY.append(mon)
      print("Opponents " + mon.name + " Destroyed")
      destroy(notPlayer, Field, opGY, mon)
    return True 
    
  #if opponent has no monsters on their field
  else:
    return False



In [ ]:
#Calvin's Cards

def SwordOfDarkDestruction(player, Field):
    # Equip only to a DARK monster. It gains 400 ATK and loses 200 DEF.

    # Most Optimal is to equip it to Player. The opponent side doesn't matter since 
    # increase the attack on player will outweight of decreasing defense in opponent side
    
    AttackIncrease = 400
    DefenseDecrease = 200   

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    dark_monsters = []
    for monster in Field[x][y]:
        if monster.attribute == "DARK":
            dark_monsters.append(monster)

    if not (dark_monsters):
        print("No DARK monsters on the field to equip the 'Sword of Dark Destruction' to!")
        return False
    else: 
        optimal_monster = max(dark_monsters, key=lambda monster: monster.total_atk)
        print("Equipping 'Sword of Dark Destruction' to", optimal_monster.name, "to increase its attack by", AttackIncrease, "and decrease its defense by", DefenseDecrease)
        optimal_monster.bonus_atk += AttackIncrease
        optimal_monster.bonus_def -= DefenseDecrease
        optimal_monster.equipped.append('Sword of Dark Destruction')
        return True
     


def BookOfSecretArts(player, Field):
   # A Spellcaster-Type monster equipped with this card increases its ATK and DEF by 300 points

    # Most Optimal is to equip it to Player monster 
    AttackIncrease = 300
    DefenseDecrease = 300   

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    spellcaster_monsters = []
    for monster in Field[x][y]:
        if monster.race == "Spellcaster":
            spellcaster_monsters.append(monster)

    if not (spellcaster_monsters):
        print("No SpellCaster monsters on the field to equip the 'Book Of Secret Arts' to!")
        return False
    else: 
        optimal_monster = max(spellcaster_monsters, key=lambda monster: monster.total_atk)
        print("Equipping 'Book Of Secret Arts' to", optimal_monster.name, "to increase its attack by", AttackIncrease, "and decrease its defense by", DefenseDecrease)
        optimal_monster.bonus_atk += AttackIncrease
        optimal_monster.bonus_def -= DefenseDecrease
        optimal_monster.equipped.append('Book of Secret Arts' )
        return True

def DarkHole(player, Field, GY, opGY):
    # Destroy all monsters on the field.
    if player == 1:
      x = 0
      y = 1
      notPlayer = 2
    else:
      x = 1
      y = 0
      notPlayer = 1

    monsterField1 = Field[x][y]
    monsterField2 = Field[y][x]

    # Return None if nothin on opponents Field
    if len(monsterField2) == 0:
        return False

    #if opponent controls monster and player doesn't, activate destroy opponents monsters
    elif len(monsterField1) == 0:
       for monster in (monsterField2):
            destroy(notPlayer, Field, opGY, monster)

    #if both player and opponent controls monster see who has strongest monster
    else:
      optimal_monster = max(monsterField1, key=lambda monster: monster.total_atk)
      op_optimal_monster = max(monsterField1, key=lambda monster: monster.total_atk)

      #if opponent has the strongest monster send all monsters to the GY
      if optimal_monster < op_optimal_monster:
        for monster in (monsterField1):
            destroy(player, Field, GY, monster)
        for monster in (monsterField2):
            destroy(notPlayer, Field, opGY, monster)

      #else don't use this card
      else:
        return False
       

    Field[0][1] = []  
    Field[1][0] = []  

    return True
    
def DianKetotheCureMaster(LP):
    # Increase your Life Points by 1000 points.
    LP += 1000
    return LP

In [ ]:
#Chase's Cards

def CastleWalls(player,Field):
    #Increase a selected monster's DEF by 500 points during the turn this card is activated.
    DefenseIncrease = 500

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    #Finds highest defense monster
    monsters = Field[x][y]
    optimal_monster = max(monsters, key=lambda monster: monster.total_def)
    tempAttribute = optimal_monster.total_def + DefenseIncrease
    #switch player field
    temp = x
    x = y
    y = temp

    #finds opponents highest attack monster
    monsters = Field[x][y]
    op_optimal_monster = max(monsters, key=lambda monster: monster.total_atk)

    #if oppenent highest attack monster points is higher than players highest defense monster points then it is useless
    if op_optimal_monster.total_atk > tempAttribute:
        return False
    else:
        print("Equipping 'Castle Walls' to", optimal_monster.name, "to increase its defense by", DefenseIncrease)
        optimal_monster.bonus_def += DefenseIncrease
        optimal_monster.equipped.append('Castle Walls')
        return True
        

def DarkEnergy(player, Field):
    #A Fiend-Type monster equipped with this card increases its ATK and DEF by 300 points.
    AttackIncrease = 300
    DefenseIncrease = 300 

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    fiend_monsters = []
    for monster in Field[x][y]:
        if monster.race == "Fiend":
            fiend_monsters.append(monster)

    if not (fiend_monsters):
        print("No FIEND-TYPE monsters on the field to equip the 'Dark Energy' to!")
        return False
    else: 
        optimal_monster = max(fiend_monsters, key=lambda monster: monster.total_atk)
        print("Equipping 'Dark Energy' to", optimal_monster.name, "to increase its attack by", AttackIncrease, "and increase its defense by", DefenseIncrease)
        optimal_monster.bonus_atk += AttackIncrease
        optimal_monster.bonus_def += DefenseIncrease
        optimal_monster.equipped.append('Dark Energy')
        return True



def Invigoration(player, Field):
    #An EARTH monster equipped with this card increases its ATK by 400 points and decreases its DEF by 200 points
    AttackIncrease = 400
    DefenseDecrease = 200   

    if player == 1:
        x = 0
        y = 1
    else:
        x = 1
        y = 0

    earth_monsters = []
    for monster in Field[x][y]:
        if monster.attribute == "EARTH":
            earth_monsters.append(monster)

    if not (earth_monsters):
            print("No EARTH monsters on the field to equip the 'Invigoration to!")
            return False
    else: 
        optimal_monster = max(earth_monsters, key=lambda monster: monster.total_atk)
        print("Equipping 'Invigoration' to", optimal_monster.name, "to increase its attack by", AttackIncrease, "and decrease its defense by", DefenseDecrease)
        optimal_monster.bonus_atk += AttackIncrease
        optimal_monster.bonus_def -= DefenseDecrease
        optimal_monster.equipped.append('Invigoration')
        return True

def SakuretsuArmor(player, Field, opGY, opAttackingCard):
    #When an opponent's monster declares an attack: Target the attacking monster and destroy it

    #only can be used when opponet is attacking, so return none if op is not
    if opAttackingCard == None:
        return False

    else:
      # set field (x, y) values to opposite players field
        if player == 1:
            x = 1
            y = 0
            notPlayer = 2
        else:
            x = 0
            y = 1
            notPlayer = 1
        op_Field = Field[x][y]
        if len(op_Field) > 0:
            
            #destroy monster on the opponents field
            count=0
            mon = opAttackingCard
            destroy(notPlayer, Field, opGY, mon)
            print(mon.name," Destroyed")
            return True
            
        #if opponent has no monsters on their field
        else:
            return False


In [ ]:
#Matt's Cards

def fissure(player, Field, opGY):
  # Destroy the one face up monster that has the lowest attack.

  if player == 1:
    x = 1
    y = 0
    notPlayer = 2
  else:
    x = 0
    y = 1
    notPlayer = 1

  if len(Field[x][y]) > 0:
    min_atk_monster = min(Field[x][y], key=lambda monster: monster.total_atk)
    destroy(notPlayer, Field, opGY, min_atk_monster)
    print(min_atk_monster.name + " Destroyed")
    return True

  else:
    return False


def trapHole(player, Field, opGY, summonedMonster):
  # Destroys target monster with 1000 or more ATK.

  if player == 1:
    x = 1
    y = 0
    notPlayer = 2
  else:
    x = 0
    y = 1
    notPlayer = 1

  monster = summonedMonster
  if monster.attack >= 1000:
    destroy(notPlayer, Field, opGY, monster)
    print(monster.name + " destroyed by Trap Hole.")
    return True
  else:
    return False


def deSpell(player, Field, opGY):
  # Target card on field; destroy that card if it is a spell card.

  if player == 1:
    x = y = 1
  else:
    x = y = 0

  if len(Field[x][y]) > 0:

    for card in Field[x][y]:
      cardType = re.search("spell", card.type.lower())
      if cardType is not None:
        spell = card
        opGY.append(spell)
        i = Field[x][y].index(spell)
        Field[x][y].pop(i)
        print(spell.name + " removed from the opponent's spell and trap zone.")
        return True

    else:
      print("Opponent has no spell cards.")
      return False


def reinforcements(player, Field):
  # Targets 1 face-up monster on the field. Adds 500 ATK until the end of the turn.

  AttackIncrease = 500
  if player == 1:
    x = 0
    y = 1
  elif player == 2:
    x = 1
    y = 0

  targets = []
  if len(Field[x][y]) == 0:
    print("There are no monsters to target on the field.")
    return False
    
  target = max(Field[x][y], key=lambda monster: monster.total_atk)
  target.bonus_atk += AttackIncrease
  target.equipped.append('Reinforcements')
  print(target.name + " gains 500 ATK until the end of this turn.")
  return True

In [ ]:
#Raj's Cards

def changeOfHeart(player, Field):
    #Targets 1 monster your opponent controls; take control of it until the End Phase.
    monsterOp = Field[1][0]
    monsterPl = Field[0][1]

    if monsterOp == 0:
        print("No monsters on the opponents field")
        return False
    else:
        #takes the optimal monster from the opponent to add it to the players monster
        optimalMonster = max(monsterOp, key=lambda monster: monster.total_atk)
        optimalMonster.switched = True
        monsterPl.append(optimalMonster)
        monsterOp.remove(optimalMonster)
        

    return True

        

def waboku(player, Field):
    #You take no battle damage this turn. Your monsters cannot be destroyed by battle this turn.
    monsterOp = Field[1][0]

    if monsterOp == 0:
        print("No monsters on the opponents field")
        return False
    
    #put all opponent monsters in defense position so they cannot attack the player as they dont take any battle damage this turn
    for monster in monsterOp:
        monster.can_atk = False
    return True



def dragonCaptureJar(player, Field):
    #Change all face-up Dragon-Type monsters on the field to Defense Position,
    #also they cannot change their battle positions.
    if player == 1:
      x = 0
      y = 1
    else:
      x = 1
      y = 0

    monsterPl = Field[x][y]
    monsterOp = Field[y][x]

    dragonMonster = []
    opDragonMonster = []
    # Return None if nothin on Field
    if  len(monsterOp) == 0:
        return False
    
    if len(monsterOp) > 0:
      
      for monster in Field[y][x]:
        if monster.race.lower() == "dragon":
          opDragonMonster.append(monster)

      if len(dragonMonster) == 0:
        return False

      else:
        if len(monsterPl) > 0:
          for monster in Field[x][y]:
            if monster.race.lower() == "dragon":
              dragonMonster.append(monster)
        if len(dragonMonster) > 0:
          opOptimalMonster = max(opDragonMonster, key=lambda monster: monster.total_atk)
          optimalMonster = max(dragonMonster, key=lambda monster: monster.total_atk)
          if opOptimalMonster > optimalMonster:
            for monster in Field[x][y]:
              if monster.race.lower() == "dragon":
                  monster.can_atk = False
          else:
            return False

        for monster in Field[y][x]:
          if monster.race.lower() == "dragon":
            monster.can_atk = False
            
   

    #Change all face up dragon monsters to defense positions

    return True





In [ ]:
letsDuel(p1Deck, p2Deck)